# Natural Language on Neural Networks: A primer
In the last class we progressively built neural networks using the little toolkit that we introduced in the first lesson. After introducing deep neural networks, that is, networks with one or more hidden layers, we played around with our toy problem and left some extra exercises to mess around with. The first one was meant to cover what we had seen more in-depth, with different datasets, inputs and model variables:

* [https://playground.tensorflow.org/](https://playground.tensorflow.org/)

The second exercise was a mental warm-up of sorts. We simply asked ourselves two questions: **What are letters, words or sentences? How do we represent those and feed them into a neural net?**

As always, we want to focus on programming as much as possible. Let's look at these two questions separately, first by trying to answer them in a simple way, then noticing our mistakes and finally sketching the whole picture. By the end of the lesson, we should be able to tackle simple problems dealing with Natural Language Processing! Let's begin by preparing a little toy example:

In [ ]:
import numpy as np

# we have a very simple text
sample_text = '''the cat sat on the mat. 
the rat saw the cat on the mat and ran.
then so did the cat.'''

# the text has lines, words, chars?
lines = [l.strip() for l in sample_text.split('.') if l]
words = set([w for l in lines for w in l.split(' ') if w])
chars = set([c for w in words for c in w])

# print total counts
print('There are {} lines.'.format(len(lines)))
print('There are {} unique words.'.format(len(words)))
print('There are {} unique chars.'.format(len(chars)))

We have to remember what we have seen so far, however. **Our neural network expects vectors of real numbers as inputs. How can we turn those characters into fixed size inputs?**

An option is to pass the ordinal value of the character. That way, each character is a single value. To keep word inputs fixed in size, we can limit the maximum length of our words, padding with a value that means *'there is nothing here'* when the word is short. Let's see how this would look like:

In [ ]:
# compute the maximum length out of all words
max_len = max(map(len, words))

# pad words with the 'null' character, \0
words_padded = [w + '\0' * (max_len - len(w)) for w in words]

# represent each word as a vector of max_len values, 
# each containing the ordinal value of the character!
words_vector = np.asarray([[ord(c) for c in w] for w in words_padded])

print('Our words in tensor form is a {} matrix:'.format(words_vector.shape))
print(words_vector)

Although straightforward, this aproach has a few issues:

* We are representing characters in an order, but each of the characters are independent. For the network, values 96 and 97 will be close. However, they represent '\`' and 'a' respectively. Coming back to the geometric intuition we learned in the first lesson, **the network will have to learn planes to separate each of the indices before it can effectively use them as inputs.**
* The neural network will have to learn to deal with the padding. Furthermore, characters that are in the last positions will have fewer examples and might not be learned well unless we train with enormous amounts of data.

In sort, discretized inputs fit poorly for a neural network. The question that follows is if we can slightly tweak the input so that our networks have no problems with them. Let's try solving the first problem in a very simple way called one-hot encoding:

In [ ]:
# one-hot encoding: 
# represent discrete values as 'on' flag in a boolean vector!
def one_hot(v, length=256):
    '''Encode a vector of indices as a one hot vector of a given length'''
    states = np.zeros(length)
    states[v] = 1
    return states

words_one_hot = np.asarray([[one_hot(ord(c)) for c in w] for w in words_padded])

print('All of our words in one-hot encoded tensor form is a {} matrix:'.format(words_one_hot.shape))
print(words_one_hot)

We have solved the first issue by turning each of our words into a binary matrix. We do so by representing every character as a vector in which the index corresponding to the character is set to 1, and 0 everywhere else. Take a second to imagine how that will look if we only had four characters a, b, c and d:

<img src="images/OneHotChars.png" width="300" alt="One-hot encoded alphabet of 4 characters"></img>

Now the question is: how do we provide this as an input to a network? The simplest approach is to **flatten a word matrix so that it is a vector instead.** A picture is worth a thousand words, so let's see how our input would look for our four-character problem if words were 3 characters each:

<img src="images/OneHotWord.png" width="600" alt="One-hot encoded word 'cab' in our 4 character alphabet"></img>

At this point, we have to picture our neural network. You should realize that on the first layer, our input would *sort of* select each of the column vectors in the underlying matrix. **A word would simply be the sum of all those column vectors, each of which would represent a character.** One of the few caveats is that character `a` at the beginning of the word would be totally different from character `a` in the middle of it. Furthermore, in a fully fledged corpus we would still have the issue of word lengths and we would have to deal with words that are longer than our limit, the fewer training examples for characters at the end of words, etc. 

Can we do better? Perhaps we are using too much information: does it make sense for us to pay attention at characters? After all, usually when we want to deal with text in a natural language, the meaning of text comes from words and how they relate with one another. **Let's simplify the problem by only considering the words that show up in each line:**

In [ ]:
# build our vocabulary
vocabulary = {}
for w in words:
    vocabulary[w] = len(vocabulary)

# get the total number of tokens
num_words = len(vocabulary)

# function to turn a vocabulary index into a one-hot array!
to_one_hot = lambda x: one_hot(x, num_words)

# represent the lines as a bag of words, padded to a maximum possible length
line_bows = [set([vocabulary[t] for t in l.split(' ')]) for l in lines]
line_matrix = np.asarray([sum(map(to_one_hot, l)) for l in line_bows])

# let's take a look at our resulting matrix
print('After indexing and padding, we get the following {} matrix representing the lines: '.format(line_matrix.shape))
print(line_matrix)

What we have implemented is known as a `Bag of Words`, often a `BoW`. It represents a document as the set of words that appear in it. To make it into something that we can feed into a neural network, we simply represent the document as a vector with as many components as there are words we consider. In our case, there are 12 words, so each line is a vector with 12 components. In doing this, we have the basic architecture for modelling an NLP task with neural networks!

Of course, realistic corpora have many more different words! In fact, general datasets across the web, for instance [wikipedia] or [common crawl] have millions of unique words. And this is precisely the problem with this approach: it might be the case that we have many different words, but not enough data to effectively train a model. 

# Sentiment Analysis: Our first NLP problem!

To take the point home, let's introduce a first instance of the real problem we will be working with: **sentiment analysis**. In short, we are given a text snippet and we want to classify whether the general sentiment of the text is positive or negative. Our data will come from IMDB, so we will be dealing with movie reviews. The original publication with the data can be found [here](http://www.cs.cornell.edu/people/pabo/movie-review-data/). Let's begin by loading the data:

In [ ]:
import io
import os

base_path = '../Datasets/Sentiment'
full_dataset = {}
for c in ['neg', 'pos']:
    class_path = '{}/{}'.format(base_path, c)
    class_docs = []
    file_names = next(os.walk(class_path))[2]
    for f in sorted(file_names):
        file_path = '{}/{}'.format(class_path, f)
        document = io.open(file_path, 'r', encoding='utf-8').read()
        class_docs.append(document)
    full_dataset[c] = class_docs
    
total_documents = sum(map(len, full_dataset.values()))
print('Loaded the dataset with {} total documents.'.format(total_documents))

#### Defining the dataset and the split sizes [[Exercise 1]](#Exercise-1:-How-does-dataset-size-affect-our-results?)
Just to make sure everything is alright, we will print one document of each type. We will also prepare a partition to evaluate our model, which will include the last 100 documents of each type. In doing so, we will start noticing some of the practical problems of our approach. Let's take a look:

In [ ]:
import random
from collections import Counter

dataset_size = 1000
train_size = 800
test_size = dataset_size - train_size

# prepare the training and test partitions
train_docs = full_dataset['pos'][:train_size] + full_dataset['neg'][:train_size]
test_docs  = full_dataset['pos'][-test_size:] + full_dataset['neg'][-test_size:]

# set the labels: first half is positive, second half is negative
train_labels = [1] * train_size + [0] * train_size
test_labels  = [1] * test_size + [0] * test_size

# set a random seed for reproducibility and shuffle the datasets
shuffle_seed = 3
for l in [train_docs, test_docs, train_labels, test_labels]:
    random.seed(shuffle_seed)
    random.shuffle(l)

# take a look at a few documents
neg_review = full_dataset['neg'][0]
pos_review = full_dataset['pos'][0]
print('>> Negative example -- first 1000 chars (out of {})'.format(len(neg_review)))
print('It has {} lines and {} total words'.format(len(neg_review.splitlines()), 
                                                  len([w for l in neg_review.splitlines() 
                                                         for w in l.split(' ')])))
print('-----------------------------------')
print(neg_review[:1000] + '...')
print('-----------------------------------')
print('>> Positive example -- first 1000 chars (out of {})'.format(len(pos_review)))
print('It has {} lines and {} total words'.format(len(pos_review.splitlines()), 
                                                  len([w for l in pos_review.splitlines() 
                                                         for w in l.split(' ')])))
print('-----------------------------------')
print(pos_review[:1000] + '...')

# compute stats & make sure that the labels are shuffled
all_docs = train_docs + test_docs
document_words = [[w for l in d.splitlines() for w in l.split(' ') if w] for d in all_docs]
word_counts = Counter([w for d in document_words for w in d]).most_common()

total_words = len(word_counts)
total_appearances = sum([v for (_, v) in word_counts])

# compute % of words that appear only once per document
document_bow_words = [w for d in document_words for w in set(d)]
multidoc_counts    = Counter(document_bow_words).most_common()
single_doc_words   = [w for w, f in multidoc_counts if f == 1]
single_doc_percent = 100.0 * len(single_doc_words) / float(total_words)

print('-----------------------------------\n\n')
print('Our first 10 shuffled training labels look like {}'.format(train_labels[:10]))
print('We have {} unique words in our dataset, in {} total word occurrences. '.format(total_words, total_appearances))
print('{0:.2f}% words appear in only one document.'.format(single_doc_percent))

We can see that the dataset has already been preprocessed for us, and that all documents are long enough. By this we just mean that we have a good amount of information to create a model. In fact, we might have too much information!

After all, **our measly 8MB dataset with only 2000 documents contains more than 50000 distinct words!** Furthermore, almost 50% of the words appear in just one document altogether. This can be a problem, because a model could just learn to match those words directly with a label and be done with it! If we look at the snippets, we can see that a few words are bound to 'multiply' themselves, such as terms with numbers (`'80s, 12-part, 500`) or apostrophes (`they're, don't, hell's`). Additionally, some words may be very specific to the review, for instance actor or character names. This alone is enough to cause some problems—let's list a few of the ones we can foresee:

* We will train on a dataset and then test our model on another—**but there will be unseen words!** We have to come up with a way to deal with those, the most simple of which is just ignoring them.
* Our previous bag-of-words formulation means that **at the very least we will be training a different vector for each of the words in the dataset**. Because we have at most 1000 documents of each class to train, this means that we will have many more variables than instances to learn from. Because of this, **our model will probably not generalize very well.**
* The data might need to be processed and cleaned again. Doing so, we can normalize numbers, apostrophes and other special cases.

Phew! We haven't even started building a model and we're already imagining issues! Since we are coders, however, we will build a model and look at the actual results—we might be wrong in our assessment! 

#### Preparing the BoW inputs [[Exercise 2]](#Exercise-2:-Frequency-inputs-for-our-Bag-of-Words.)
Let's prepare the BoW inputs and then create the simplest possible model, a shallow neural network:

In [ ]:
# encoding vocabulary and the encoding function
word_indices = {t[0]: i for i, t in enumerate(word_counts)}
inv_word_idx = {i: t[0] for i, t in enumerate(word_counts)}

def prepare_data(docs, labels):
    '''Vectorize documents and labels so they can be used as input for a neural network'''
    tokenized_docs = [set([word_indices[w] for l in d.splitlines() 
                                           for w in l.split(' ') if w]) 
                          for d in docs]
    X = np.asarray([one_hot(list(d), total_words) for d in tokenized_docs])
    y = np.asarray(labels)
    return X, y

X_train, y_train = prepare_data(train_docs, train_labels)
X_test,  y_test  = prepare_data(test_docs, test_labels)

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense

num_epochs = 10

# define the layers
inp = Input((total_words,), name='input_bow')
out = Dense(1, name='output_f', activation='sigmoid')(inp)

# define and compile the model -- we use a different optimizer this time!
sent = Model(inputs=inp, outputs=out)
sent.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
sent.summary()
sent.fit(X_train, y_train, epochs=num_epochs, batch_size=64, verbose=1, validation_data=(X_test, y_test))

Well, that's a nice surprise! Our network managed to completely overfit the training data in 10 training epochs and it still performed reasonably well in our validation split! **In particular, you should be getting a 100% accuracy on our training set and around 87% on the validation set.**

If you play around with the number of epochs, you will notice that the network keeps on learning, reducing the `val_loss` (validation loss) over time. This means that it keeps improving on unseen validation data as it goes on minimizing the error, the training loss. However, we have to ask ourselves: **what have we actually learned?** Let's play with our model to try and understand what it's doing:

In [ ]:
word_weights = sent.get_weights()[0]
words_top = 2000
words_to_show = 50

# get the learned weights for the top words
topn_weights = word_weights[:words_top].flatten()
topn_sorted  = topn_weights.argsort()
topn_neutral = np.abs(topn_weights).argsort()

# get the top words
topk_pos = [inv_word_idx[i] for i in topn_sorted[-words_to_show:]][::-1]
topk_neg = [inv_word_idx[i] for i in topn_sorted[:words_to_show]]
topk_neu = [inv_word_idx[i] for i in topn_neutral[:words_to_show]]

# print 'em all
print('The top {} positive tokens taken from the {} most frequent words are: \n\n — {}\n'.format(words_to_show, 
                                                                                                   words_top, 
                                                                                                   ', '.join(topk_pos)))
print('The top {} negative tokens taken from the {} most frequent words are: \n\n — {}\n'.format(words_to_show, 
                                                                                                   words_top, 
                                                                                                   ', '.join(topk_neg)))
print('The top {} neutral tokens taken from the {} most frequent words are: \n\n — {}\n'.format(words_to_show, 
                                                                                                  words_top, 
                                                                                                  ', '.join(topk_neu)))

Let's unpack that a bit, though the results seem convincing! Our model takes a Bag of Words input representing all the possible words that may exist in a document as a binary vector. Words that appear, no matter the frequency, are a 1 in our input. Words that do not are a 0. Then we try to predict a single binary variable, the sentiment of the review, which is either positive (1) or negative (0). 

The network is shallow. There are no hidden layers, and we just learn one weight for each of the inputs. Since the inputs are binary, the weight will mean the word is either **positive evidence** or **negative evidence** depending on the sign. If we get our weights matrix, we can simply check how positive or negative a word is perceived to be! If we sort the words according to their weight, we can see the words that are more highly positive or negative. We can also draw a histogram of the overall distribution of weights:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(9, 6), dpi=120, facecolor='w', edgecolor='k')
plt.subplot(2, 2, 1)
plt.title('Distr. of sentiment word-weights (all words)')
plt.xlabel('Sentiment weight')
plt.ylabel('# of words')
hist = plt.hist(word_weights.flatten(), bins=50)

plt.subplot(2, 2, 2)
plt.title('Distr. of sentiment word-weights ({} most freq.)'.format(words_top))
plt.xlabel('Sentiment weight')
plt.ylabel('# of words')
hist = plt.hist(topn_weights, bins=50)

plt.tight_layout()

all_mean, topn_mean = word_weights.mean(), topn_weights.mean()
print('The average weight for a word is: {} on the full dataset and {} on the most frequent words.'.format(all_mean, topn_mean))

Most words seem to have little or no impact at all, with weights very close to 0. This, of course, makes sense for this task. After all, sentiment analysis is a task in which just a few words tell us the most: `awful, great or predictable` are great examples of this. Of course, our model is very simplistic and we have to remember that this dataset is small by today's standards. And from that, we can have some reasonable doubts. In particular: 

* Our methodology is very brittle! **What happens with words we have never seen?**
* And some words, we have only seen once... **What if the weights we got for them are wrong?**
* The more words, the larger our problem becomes... **What if our model needs to accomodate hundreds of thousands, millions of different words?**
* Is it even sufficient for the problem? **We are not considering relationships between words!**

These questions are yet another challenge we have to overcome. After all, not all natural language problems are going to be this simple! In some cases, we might need our model to understand the structure of our sentence or paragraph. In those cases, we will need more advanced machinery, and their corresponding weights—and our datasets may not be large enough to learn them like in this case!

Consider, for instance, the problem of **Question Answering**. There are multiple ways of defining it, but one of them will make our points very clear. Let's say we have a question and a paragraph that contains an answer to that question. Our task is to find the range of text from the paragraph that corresponds to the answer. To do this, **we clearly need to capture relationships between words. Furthermore, we need a model that learns relationships across pieces of text!** 

Naively, we could try to manage this by extending our input to 'groups of words'. This means that our input will be pairs, triples or any arbitrary number of words clumped together: this is generally called **ngrams**, where n is the number of words. Let's see what this would mean for our toy sentiment analysis task:

In [ ]:
def make_ngrams(tokens, n):
    '''Make n-grams out of a list of tokens, for a given n.'''
    return [' '.join(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]

# run through all the possible ngrams from 1 to 5
max_ngram_tokens = 5
num_ngram_tokens = range(1, max_ngram_tokens + 1)
freq_counts = []
for n in num_ngram_tokens:
    document_ngrams = [make_ngrams([w for l in d.splitlines() for w in l.split(' ') if w], n)  for d in all_docs]
    ngram_counts = Counter([w for d in document_ngrams for w in d]).most_common()

    # compute stats for the ngrams
    max_freq = ngram_counts[0][1]
    total_ngrams = len(ngram_counts)
    top_ngrams = ', '.join(['"{}"'.format(t) for t, f in ngram_counts[:10]])
    
    # store maximum frequency and print
    freq_counts.append(max_freq)
    print('There are {} total ngrams with n={}, the most common of which ocurring {}'
          ' times and with the 10 most frequent bigrams being: {}.\n'.format(total_ngrams, n, max_freq, top_ngrams))

# plot the frequency counts given the tokens per ngram
plt.title('Maximum ngram frequency vs. tokens per ngram')
plt.xlabel('Tokens per ngram')
plt.ylabel('Maximum ngram frequency')
plt.semilogy(num_ngram_tokens, freq_counts)
plt.xticks(num_ngram_tokens)

print('Note: the y axis is logarithmic!')

As we go on making our ngrams longer and longer, they resemble meaningful sentences more and more. However, it is clear that the ever-decreasing frequencies will make our model have very few samples, and training will become ever-increasingly harder without enormous datasets. After all, we can't train on what we can't see!

Let's get some perspective, as this approach feels like obvious brute-force. Such brute-force actually powered most of NLP before the advent of those neural models, with what was called **probabilistic language models**. The idea was, in short, to have a model that could tell us the probability of a given sequence of tokens, or the probability of one specific token given a sequence of previous ones. 

This idea, in and out of itself, makes sense: the problem is that for a long time we approached it by simply computing the probabilities of ngrams altogether. Count-based models needed huge amounts of data and never managed to fully capture the underlying relationships between the words. Of course, **we are speaking in the general sense: we have seen that some problems can be solved with just that!** However, for problems similar to machine translation, the results were notoriously bad.

What can we do? First of all, the idea of a probabilistic language model is not all that bad: if we could find a way to capture such relationships in a compact way, perhaps it could simplify the input for many NLP tasks! In fact, let's put it that way: **can we solve a single task that helps us in all NLP tasks?** 

In the simplest case, we would want to know if a word should appear given another one. That is, given a word, we would want to classify which words could follow. Jumping into code, we could do something like this:

In [ ]:
# our input is a word, our target the words afterwards
all_words = [w for d in document_words for w in d] 

print('The total number of words to process is {}.'.format(len(all_words)))

In [ ]:
# Q: why did we comment out the code to create the data?
# Hint: what will be the resulting size of X and y?
# Hint: does that fit in memory?
#X_one = np.asarray([one_hot(word_indices[w], total_words) for w in all_words[:-1]])
#y_one = np.asarray([one_hot(word_indices[w], total_words) for w in all_words[1:]])

# define the layers
inp = Input((total_words,), name='input_word')
out = Dense(total_words, name='output_word', activation='softmax')(inp)

# define and compile the model -- different loss function as we can have multiple outputs!
all_to_all = Model(inputs=inp, outputs=out)
all_to_all.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
all_to_all.summary()

# Q: why didn't we even try to train the model?
# Hint: look at the number of parameters!
#all_to_all.fit(X_one, X_one, epochs=10, batch_size=64, verbose=1) 

# Clean up 2.6B parameters just to be eco-friendly
del all_to_all

Our little model, **another shallow neural network with no hidden layers, has up to billions of parameters!** Well shoot, that's going to be hard to train! Though of course, what else were we expecting? We are trying to model the relationship between any of the $N$ words with any of the other $N$ words, so of course our resulting matrix is going to be $N \times N$!

Now imagine if we tried to also include ngrams! We would be representing each of the input ngrams as a one-hot vector, which means that each input has as many components as there are ngrams in our dataset. **With just the unigram and bigram counts that we have seen, this means more than half a million vectors!** 

In actuality, since our input is going to be a single one-hot value, we could compute the representation as it goes using a generator, or let Keras handle the conversion. The overhelming amount of zeros in vectors, known as sparse vectors, is a problem. However, for inputs it is fine because the library can handle it. There is little we can do with the enormous matrix that makes up our neural network! Can we do better? Let's try adding a small hidden layer, which will let us learn smaller matrices going in and going out:

In [ ]:
# we just worry about the words for now and will send one hots as we train
X = np.asarray([w for w in all_words[:-1]])
y = np.asarray([w for w in all_words[1:]])

# generator function to pass one-hot encoded vectors
gen_batch_size = 1000
num_batches = np.ceil(float(dataset_length) / gen_batch_size) 
def data_gen(X, y, batch_size, interrupt_at=None):
    '''Generator to shuffle and produce batches over a given dataset, 
    encoding samples as they are fed into the network.'''
    while True:
        random.seed(shuffle_seed)
        random.shuffle(X)
        random.seed(shuffle_seed)
        random.shuffle(y)
        for i in range(0, len(X), batch_size):
            if i == interrupt_at * batch_size:
                raise Exception("Interrupted so you don't have to wait forever!")
            
            X_in = np.asarray([to_one_hot([w]) for w in X[i: i+batch_size]])
            y_in = np.asarray([to_one_hot([w]) for w in y[i: i+batch_size]])
            yield X_in, y_in


# if we don't interrupt, it will take hours or even days!
generator = data_gen(X, y, gen_batch_size, interrupt_at=4)

# define the layers
inp = Input((total_words,), name='input_word')
hid = Dense(20, name='hidden_layer')(inp)
out = Dense(total_words, name='output_word', activation='softmax')(hid)

# define and compile the model -- different loss function as we can have multiple outputs!
with_hidden = Model(inputs=inp, outputs=out)
with_hidden.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
with_hidden.summary()

# remove comments to try to see how long it takes
#try:
#    with_hidden.fit_generator(generator, steps_per_epoch=num_batches, epochs=1, verbose=1) 
#except Exception:
#    print('Left early because this model will take too long to train!')

# free memory because the model is massive
del with_hidden

Nice! We have gone from completely untractable to painfully slow! The underlying idea here is that we use a hidden layer to encode the relevant information of a word and then, hoping that it suffices, we try to predict the next word. **The network builds a representation that encodes the information about our problem in a compressed manner.** When a network is used to learn an encoding in this way, we usually say it is an instance of **representation learning**. Those compressed representations at times help us in other tasks—perhaps this is the mythical one-task-to-rule-'em-all we are looking for!

The problem is that this approach is still limited by how slow it is. In particular, the many possible outputs make the problem very hard to learn, as we have to tune weights for each of the possible output words. Since we want a good representation of words no matter what, it seems like a waste of effort to create such representation in both directions. **Other than that, we are doing a somewhat poor job at modelling words: does it make sense to only look at the next word?** 

Let's start from the end: what is forcing us to just take the word afterwards? Instead, we can take a few words around a given word, and try to predict any of those. A common saying in linguistics is that `you shall know a word by the company it keeps`—and knowing words we will! Let's improve our generator to take care of that:

In [ ]:
def compute_windows(words, mapping={}, window_size=5):
    '''Compute word windows given a sequence of tokens'''
    centers = []
    windows = []
    for i, w in enumerate(words):
        centers.append(mapping[w] if mapping else w)
        window = words[max(0, i-window_size):i] + words[i+1:i+1+window_size]
        windows.append([mapping[t] if mapping else t for t in window])
    return centers, windows

window_size = 5
tokens_to_show = 40
X, y = compute_windows(all_words, window_size=window_size)

print('First {} tokens in the corpus and their corresponding windows:'.format(tokens_to_show))
print('-----------------------------------')
print(' '.join(all_words[:tokens_to_show + window_size]))
print('-----------------------------------\n')
print('{0: >14} : {1}\n'.format('token', 'tokens in the window for the token'))
for c, w in zip(X[:tokens_to_show], y[:tokens_to_show]):
    print('{0: >14} : {1}'.format(c, ', '.join(['"{}"'.format(t) for t in w])))
print('\n-----------------------------------')

We compute nearby words in a window as we go on through our corpus. In effect, we are trying to capture all the words in our *context* so that for a given word we can predict whatever is nearby. This makes our model more meaningful, but we have done nothing to deal with the performance issues it has! 

Let's consider two slightly different tasks. First, **can we predict the word given a context window of the words around them?** In a similar fashion, **can we predict whether a word is likely to appear in a context window or not for a given word?** The following figure gives us an intuition of the two problems we are discussing:

<img src="images/WordContextModels.png" width="600" alt="The two proposed tasks: predicting words given a context and checking for target-to-context co-occurrence."></img>

Keep this idea in mind and remember the road we have travelled up to this point. It is time to introduce the cornerstone of neural natural language processing: **word embeddings**!

# Word Embeddings: Neural Representation Learning for NLP!

Up to this point we have been improving from the very barebones model we started with. We have come far from the Bag of Words that we initially tried to train, the exploding number of ngrams that we saw in front of our eyes, the probabilistic language models that try to capture word-to-word relationships...

Word embeddings are the application of those ideas we have been playing with—just a few more strokes and we will have our full picture! The first fundamental idea is what we did to reduce our number of parameters: learn a compressed representation of a word that does a good enough job. **This means learning a $N \times d$ matrix so that each of the $N$ tokens has an associated vector of length $d$.** Converting words into this dense vectors is why we call them word embeddings, because we embed a word into space, as a point.

The second idea is that we can try to capture the words in the surroundings of a word. These words will form the overall context for a word. If we can either model which words will appear given a context, or which contexts are likely given a word, we have built ourselves a neural language model! **The problem is, as we have seen, that this problem takes very long to train because of the large amount of possible outputs, which include the whole vocabulary.** If we can't effectively train while attempting to predict the following word, our fancy model isn't really helpful—it will either remain a toy or take exceptionally long to train.

If the problem is how we are structuring the target for our task, if it is the large amount of possible outputs that is killing us, what can we do? Let's first try putting into code the image that we have so far:

In [ ]:
pairs = [(w, k) for (w, c) in zip(X, y) for k in c]
X_ctx = [w for (w, k) in pairs]
y_ctx = [k for (w, k) in pairs]
total_pairs = len(pairs)

# build the generator, now taking every input
pair_gen = data_gen(X_ctx, y_ctx, gen_batch_size, interrupt_at=4)
pair_batch_size = 1000
num_pair_batches = np.ceil(float(total_pairs) / pair_batch_size) 

# define the network architecture
inp = Input((total_words,), name='input_word')
hid = Dense(20, name='hidden_layer')(inp)

# this is what is killing us!
out = Dense(total_words, name='output_word', activation='softmax')(hid) 

# define and compile the model -- different loss function as we can have multiple outputs!
with_pair = Model(inputs=inp, outputs=out)
with_pair.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
with_pair.summary()

# remove comments to try to see how long it takes
#try:
#    with_pair.fit_generator(pair_gen, steps_per_epoch=num_pair_batches, epochs=1, verbose=1)
#except Exception:
#    print('Left early because this model will take too long to train!')

# free memory because the model is massive
del with_pair

We face issues because we stubbornly insist on trying to model a probabilistic language model. To do so, we have to discover the underlying relationships between the words—that is precisely the task! Instead, let's simplify the problem by turning our picture around. We asked ourselves if:

1. **Is it possible to predict the current word given the window of words around it?**
2. **Is it possible to predict words in the context given the current word?**

One possible way of doing this is simply deciding that the output does not need to be the whole vocabulary. In a sense, both questions can be understood as yes/no questions. In the first case, we just need to find a way to merge the representation of the context and the target word, and **classify whether or not the word is expected for this context.** In the second one, we just have to **ask if the target word has co-occurred with another word that may be the context or not.**

These two questions are implemented as two different models, respectively called `Continuous Bag of Words`, or `CBoW`,  and `Skipgram`. We are not going to go into the implementation details of each of them, and instead will focus on the second one for simplicity. In the case of `Skipgram`, we don't need to model the combination of words in the context. However, in both cases there is a lingering question: **if we are turning the problem into a yes/no question, which words will we consider a *no*?** After all, our current dataset only covers words and contexts that appear together!

#### Negative sampling [[Exercise 3]](#Exercise-3:-The-effect-of-Negative-Sampling.)

This is the last piece of the puzzle. The question is, in fact, quite simple: we include artificial negative examples in our dataset. Since our vocabulary is very large, the simplest way of doing this is to randomly take words as negative cases. This technique, called **Negative Sampling**, is best explained with some code:

In [ ]:
def make_data_binary(pairs, word_dict, negative_ratio=9):
    '''Create the binary skipgram dataset as pairs of <target, context> 
    words and their labels in word contexts, with a given ratio of 
    positive to negative samples.'''
    num_pairs = len(pairs)
    vocabulary_size = len(word_dict)
    print('Generating skipgram pairs for a dataset with {} tokens.'.format(num_pairs))
    
    # build the whole dataset of random and window samples
    indexed_pairs = [(word_indices[w], word_indices[t]) for (w, t) in pairs]
    full_pairs = indexed_pairs
    for i in range(negative_ratio):
        print('Generating group {} of negative samples out of {}.'.format(i + 1, negative_ratio))
        negatives = ((t[0], x) for (t, x) in 
                               zip(indexed_pairs, 
                                   np.random.randint(0, vocabulary_size, num_pairs)))
        full_pairs.extend(negatives)
    binary_labels = [1] * num_pairs + [0] * (num_pairs * negative_ratio)
    return np.asarray(full_pairs), np.asarray(binary_labels)

# seed our training so we get the same dataset
np.random.seed(7)
X_bin, y_bin = make_data_binary(pairs, word_indices)

print('Word pairs samples and their binary label:\n')
print('\n'.join(['\t"{}", "{}": {}'.format(inv_word_idx[p[0]], inv_word_idx[p[1]], l) for (p, l) in zip(X_bin[:10],  y_bin[:10])]))
print('\n'.join(['\t"{}", "{}": {}'.format(inv_word_idx[p[0]], inv_word_idx[p[1]], l) for (p, l) in zip(X_bin[-10:], y_bin[-10:])]))

Now that we have redefined our problem, we have to think about what we want our network to do. We want to be comparing pairs of words, and then assigning a label to them. If we start looking at the architecture from its input, we can see that we will find a challenge: **our one-hot encoded inputs mean that we will add the two vectors corresponding each of the words.** This may be a viable approach to represent the task, but it also means that we will have to learn additional weights to predict our label from the vector resulting from the sum.

**An alternative is to build a layer that works as a 'look-up table' of sorts. Given an index, we will extract the vector associated with it.** Doing this, we can build a network in which the pair of words are distinct inputs, and we can use any arbitrary function to compute their similarity. 

Now recall the first lesson: **we can use the dot product to compute how much 'in-front' a point is with respect to a plane.** What if we considered the word vectors as plane vectors and points at the same time? Then, by using the dot product, we would see how well they align with one another! You can get a geometric intuition of an angle between two words $w_1$ and $w_2$ relative to the origin point here:

<img src="images/DualDotProduct.png" width="500" alt="Angle between two word embeddings"></img>

The softer, thicker lines represent also the planes placed at the origin point that the words form. Through this view, it is easy to notice that **we are training many logistic regressions, one for each word, in which planes and points nudge eachother!**

This is because the dot product actually encodes the cosine between two vectors, which gives us a sense of the angle between them. We are not going to go into the algebraic formulas, but **the general idea is that it tells us how similar the two word vectors are.** In particular, the normalized dot product will be $1$ if the vectors have the same direction, $-1$ if they have the complete opposite direction and $0$ if they are perpendicular to one another. Here, normalized means that we ignore the scale of the vector and only keep the direction—**we only have to divide a vector by its length to normalize it.** 

**By using the dot product, we simply have to turn its output into our binary feature and our model will be ready.** As we know, we only need a single unit output with a sigmoid activation for that! Let's implement what we have described, using the data we generated before as its training input:


**NOTE:** Training a word embeddings model like this take some time if you don't have a CUDA-compatible GPU. On a late 2015 MacBook Pro, it takes half an hour of training time with 5 epochs.

In [ ]:
from keras.layers import Embedding, Dot, Flatten

# define the network architecture, starting with a _shared_ embedding layer!
embedding_size = 25
emb = Embedding(total_words, embedding_size, name='embeddings')

tgt = Input((1,), name='input_target')
ctx = Input((1,), name='input_context')

# get the embeddings of each word, flattened
flat = Flatten(name='flatten_embeddings')
tgt_emb = flat(emb(tgt))
ctx_emb = flat(emb(ctx))

# compute the dot product of the two vectors 
# it is the last axis because the first one refers to the position in the batch!
dot = Dot(axes=-1, name='embedding_dot')([tgt_emb, ctx_emb])

# we now have a single output, and learn over word pairs alone!
out = Dense(1, name='output_label', activation='sigmoid')(dot) 

# define and compile the model -- different loss function as we can have multiple outputs!
emb_model = Model(inputs=[tgt, ctx], outputs=out)
emb_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
emb_model.summary()

# seed our training so we get the same results
# note: if you are using a different backend this may not work!
np.random.seed(1)
from tensorflow import set_random_seed 
set_random_seed(2)
emb_model.fit([X_bin[:, 0], X_bin[:, 1]], y_bin, batch_size=25000, epochs=5, verbose=1) 

As we said, the embedding layer is simply a matrix from which we fetch vectors as needed. Let's extract it and find words that are similar to a small test set. Because our dataset is too small to learn meaningful embeddings and we have constrained our vectors greatly, we will test with words related to movies or that give an opinion. **That is, words we would expect to appear in a movie review.** In particular, we will test:

1. script
2. terrible
3. movie
4. character
5. good
6. protagonist

Then we will promptly discuss how to get good embeddings if we don't have enough data—which is most likely the case! Following through:

In [ ]:
emb_matrix = emb.get_weights()[0]
norm_matrix = emb_matrix / np.linalg.norm(emb_matrix, axis=-1, keepdims=True)

words_to_test = ['script', 'terrible', 'movie', 'character', 'good', 'protagonist']
indices_of_words = [word_indices[w] for w in words_to_test]

word_vectors = norm_matrix[indices_of_words]
closest_vectors = word_vectors.dot(norm_matrix.T).argsort(axis=-1)[:, -7:-1][:, ::-1]

for t, i in zip(words_to_test, closest_vectors):
    print('{} is nearest to:\n\n  — {}\n'.format(t, ', '.join([inv_word_idx[idx] for idx in i])))

The nearest words to our embeddings using the normalized dot product make some sense. Of course, our dataset is small and we have built a very simple toy model—**word embeddings are trained using all sorts of tricks to improve performance!** Still, these results are not terrible.

We will build a model using those embeddings and then discuss **pretrained embeddings**, that is, embeddings trained on large text corpora to try to capture the general characteristics of how words appear with one another!

To build our sentiment model, we will have to input a representation of our document. There are many ways of doing this, but the simplest is actually similar to the `Bag of Words` representations that we saw before. **In short, we represent a document as the vectors of all the words contained in it, which we then average.** This approach is called `Doc2vec` and in many cases, the averaged representation tends to capture the nature of the document well enough. However, just as with BoW models, it does not consider order or relationships between words. Let's give it a shot:

In [ ]:
def prepare_embedded_data(docs, labels, word_dict, emb_matrix):
    '''Vectorize documents and labels so they can be used as input for a neural network,
    converting documents into the averages of each of the word embeddings taken from the embedding matrix.'''
    tokenized_docs = [[word_dict[w] for l in d.splitlines() 
                                    for w in l.split(' ') if w]
                       for d in docs]
    X = np.asarray([emb_matrix[d].mean(axis=0) for d in tokenized_docs])
    y = np.asarray(labels)
    return X, y

X_emb_train, y_emb_train = prepare_embedded_data(train_docs, train_labels, word_indices, emb_matrix)
X_emb_test,  y_emb_test  = prepare_embedded_data(test_docs, test_labels, word_indices, emb_matrix)

In [ ]:
inp = Input((embedding_size,), name='input_doc_emb')
hid = inp
num_hidden = 3
num_units_per_layer = 30
for l in range(1, num_hidden + 1): 
    hid = Dense(num_units_per_layer, name='hidden_{}'.format(l), activation='tanh')(hid)
out = Dense(1, name='output_f', activation='sigmoid')(hid)

# define and compile the model -- we use a different optimizer this time!
our_embs_sent = Model(inputs=inp, outputs=out)
our_embs_sent.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
our_embs_sent.summary()
our_embs_sent.fit(X_emb_train, y_emb_train, epochs=100, batch_size=64, verbose=1, validation_data=(X_emb_test, y_emb_test))

Well, all that just *sort of* paid off! **We have managed to learn a model that crudely approximates the task, with around 70% accuracy on the validation set!** Now, the problem is that our vectors have too few dimensions and have been trained on too little data. 

An alternative is to load a pretrained embedding matrix. **There are a few freely available pretrained models, most notably those using [word2vec](https://code.google.com/archive/p/word2vec/), [GloVe](https://nlp.stanford.edu/projects/glove/) or [FastText](https://fasttext.cc/docs/en/english-vectors.html)**. Each of them have been produced with different algorithms, and it is a good idea to check them out and try to discover the differences among them. However, we will stick with FastText since they provide embeddings for many other languages and frequently retrain them.

For context, our dataset only contained a little over 1.5 million tokens. In comparison, FastText is trained on the Common Crawl corpus, which contains **600 billion tokens coming from text all over the web**. Additionally, FastText includes sub-word information when training. Our toy model was simply a show of concepts, but usually you will want to use a battle tested model that better captures what we want!

#### Loading and using embedding matrix [[Exercise 4]](#Exercise-4:-Alternative-word-embeddings.)
We simply have to load the embedding matrix and set it all up to go. Our code, then, becomes:

In [ ]:
matrix = []
with io.open('../Embeddings/FastTextEmbeddings.txt', 'r', encoding='utf-8') as fp:
    for l in fp:
        toks = l.strip().split(' ')
        matrix.append([float(x) for x in toks[1:]])
ft_emb_matrix = np.asarray(matrix)
ft_embedding_size = ft_emb_matrix.shape[-1]

X_ft_train, y_ft_train = prepare_embedded_data(train_docs, train_labels, word_indices, ft_emb_matrix)
X_ft_test,  y_ft_test  = prepare_embedded_data(test_docs, test_labels, word_indices, ft_emb_matrix)

In [ ]:
inp = Input((ft_embedding_size,), name='input_doc_emb')
hid = inp
num_hidden = 3
num_units_per_layer = 50
for l in range(1, num_hidden + 1): 
    hid = Dense(num_units_per_layer, name='hidden_{}'.format(l), activation='tanh')(hid)
out = Dense(1, name='output_f', activation='sigmoid')(hid)

# define and compile the model -- we use a different optimizer this time!
ft_sent = Model(inputs=inp, outputs=out)
ft_sent.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
ft_sent.summary()
ft_sent.fit(X_ft_train, y_ft_train, epochs=100, batch_size=100, verbose=2, validation_data=(X_ft_test, y_ft_test))

In [ ]:
# evaluate on a few made up reviews to test our model!
test_dataset = [
    '''this flick is garbage because the protagonist hardly manages to engage 
the viewer with its really poor acting - avoid it if you can !''', # neg
    '''fantastic movie by the same director that brought us many other comedy classics''', # pos
    '''although it is not so great at times , it is decent enough 
for a long sunday evening if you have nothing else to do''', # neutral-pos
    '''the action is not bad but the script just drags on and on and 
in the end it is not enjoyable enough to justify watching it''' # neutral-neg
]
test_labels = [0, 1, 1, 0]

# prepare the dataset again
X_ours_test, y_ours_test = prepare_embedded_data(test_dataset, test_labels, word_indices, ft_emb_matrix)

# predict with our trained model
output_probabilities = ft_sent.predict(X_ours_test) # get our class probabilities between 0 and 1
predicted_classes = output_probabilities.round().flatten() # rounding gives us the nearest class!
preds_as_ints = [int(c) for c in predicted_classes]

# show the predictions and the true label
correct_preds = len([1 for (t, p) in zip(test_labels, preds_as_ints) if t == p])
class_names = ['neg', 'pos']
print('We classified our test sentences as follows: \n')
for (t, p, s) in zip(test_labels, preds_as_ints, test_dataset):
    print('"{}"\n\n\tExpected: {}\n\tPredicted: {}\n'.format(s, class_names[t], class_names[p]))
print('We classified {} out of {} test sentences correctly.'.format(correct_preds, len(test_labels)))

Our representation of a document meant that we still lose information about individual words. **Our model using FastText embeddings works well enough for our limited dataset, with validation accuracy around the 80% mark.** However, if we remember our first model using a Bag of Words input, it is clear that we could achieve better performance by operating with each of the words and then aggregating over the result. **However, to capture this relationship we will also need more data as we no longer have a clear identifier for each word!**

The model we have created works with our made up reviews, at least well enough. Because our training runs are
randomized, the two neutral cases are likely be missclassified. **To better understand the opacity of our model, it can be illuminating to play around with the words so we can get the predictions to flip!**

In the next lesson we will explore more advanced neural network architectures. **So far, our networks always take fixed inputs, always of a fixed length.** However, there are cases in which information shows some structure and it is that structure that actually helps towards solving the problem—consider the relationships of words in a sentence! 

# Getting our hands dirty: Exercises.
---

### Exercise 1: How does dataset size affect our results?
> Take a look at the work we have done so far with sentiment analysis: what if we split the data differently?
> 
> To improve your intuition about how the volume of data affects the results, try running the cells reserving different amounts of samples for the training and validation/test sets. For example, try an even split and then heavily bias it towards training.
> 
> **HINT:** You can do this by setting different values of `train_size` between 0 and 1000. You can do this [here](#Defining-the-dataset-and-the-split-sizes-[Exercise-1]).

---

### Exercise 2: Frequency inputs for our Bag of Words.
> Our Bag of Words inputs represent each of the input words as a separate binary component. A trivial extension is to replace this binary feature with the frequency counts for each of the words in the vocabulary. 
> 
> This change will affect the learned weights for each of the words. Modify the code accordingly and check if there is a noticeable effect. Does it have an effect if you also change the size of your dataset?
> 
> **HINT:** You can do this by modifying the `prepare_data` function. You can do this [here](#Preparing-the-BoW-inputs-[Exercise-2]).

---

### Exercise 3: The effect of Negative Sampling.

> To train our model, we have pre-set a few values to educated guesses. One of the more important parameters when training a word embedding model using Negative Sampling is the ratio of positive to negative examples. If this ratio is too low, the model may not learn a useful representation at all!
>
> Try preprocessing the dataset with different ratios of negatives to positives. What effect does it have on the embeddings? What effect does it have on the performance of our model and the time it takes to train it?
>
> **HINT:** Simply change the `negative_ratio` parameter when producing the dataset. You can do this [here](#Negative-sampling-[Exercise-3]).

---

### Exercise 4: Alternative word embeddings.
> We have only used FastText embeddings as they are frequently updated and readily available. However, when designing a real world system, it is often a good idea to try different embedding datasets to evaluate which one better fits our data.
> 
> This is a somewhat open exercise, but you may find the following github page useful: [Awesome embedding models](https://github.com/Hironsan/awesome-embedding-models#datasets)
>
> In particular, try to:
> 1. Play with embeddings that have a different number of components. Our dataset is very small, so it might be the case that a more condensed representation achieves better performance as we have less parameters to fit.
> 2. The FastText embeddings we used dealt with out of vocabulary for us—what if you don't have sub-word information? What is the effect in model performance when we throw unseen words away?
> 3. Have fun! Word embeddings are fascinating because they capture meaning between words with a very simple mechanism. You can play around with them and perhaps develop a little toy project on the side.
>
> **HINT:** You can do this by implementing your own loading function (or changing the loading path, if your embedding format is the same) and embedding the data. You can do this [here](#Loading-and-using-embedding-matrix-[Exercise-4]).

In [ ]:
# Exercise 1 solution: try different values e.g. [500, 750, 900]

# Exercise 2 solution:
def prepare_data(docs, labels):
    tokenized_docs = [[word_indices[w] for l in d.splitlines() 
                           for w in l.split(' ') if w]
                           for d in docs]
    X = np.asarray([np.bincount(d, total_words) for d in tokenized_docs])
    y = np.asarray(labels)
    return X, y

# Exercise 3 solution: try different values e.g. [1, 4, 9, 19]

# Exercise 4 solution: up to you and your creativity!